# Drone detection video benchmark (YOLO) — baseline + selectable ROI, motion mining, continuity gates, and double-pass verification
This notebook runs a **5 FPS (configurable)** inference policy on a video, logs results, and exports an annotated output video.

Key properties:
- **Every video frame** gets an overlay (no flicker).
- Inference happens only on inference frames; intermediate frames reuse the last inference status.
- Motion-based ROI mining runs **only after a configurable NO-DRONE streak**.
- Guided ROI for the *next* inference (tracking) is separate from confirmation logic.
- Confirmation uses a common **K-out-of-N window** with selectable continuity gating.
- Optional verification uses the **same YOLO weights** via `VERIFY_MODE`.


## 1) Imports
Loads OpenCV, NumPy, Pandas, and Ultralytics YOLO.

In [1]:
import os
import time
import math
import csv
from dataclasses import dataclass
from collections import deque, defaultdict
from typing import List, Tuple, Optional, Dict

import cv2
import numpy as np
import pandas as pd

from ultralytics import YOLO

# Optional: OpenCV contrib bgsegm (not always installed)
HAS_BGSEGM = hasattr(cv2, "bgsegm")
print("OpenCV version:", cv2.__version__)
print("cv2.bgsegm available:", HAS_BGSEGM)

OpenCV version: 4.12.0
cv2.bgsegm available: False


## 2) Configuration (your real paths + knobs)
### What the confidence parameters mean
- `BASE_CONF_FULL`: confidence threshold used for **full-frame** inference and **motion ROI** inference.
- `GUIDED_ROI_CONF`: confidence threshold used for **guided ROI** and **verification passes** (usually higher).
- `LOW_CONF_DETECTION`: if a detection is below this, it is treated as **low-trust** and (optionally/mandatorily) verified.

### Baseline
Setting `PRE_MOTION_METHOD = 'None'` and `VERIFY_MODE = 'None'` gives baseline behavior.
**Important:** baseline selection does **not** forcibly disable toggles; it only means the selected method is `None`.


In [2]:
# --- Your real paths (kept from your latest uploaded notebook) ---
MODEL_WEIGHTS = r"S:\IntelliJ\Projects\ES_Drone_Detection\runs\detect\yolo26\drone_finetune_full_mixed\weights\best.pt"
VIDEO_PATH    = r"S:\IntelliJ\Projects\ES_Drone_Detection\video_test\gopro_006.mp4"

# Annotation path: prefer your local file; else use the uploaded example in this environment.
ANNOTATION_PATH_PRIMARY  = r"S:\IntelliJ\Projects\ES_Drone_Detection\video_test\gopro_006.txt"
ANNOTATION_PATH_FALLBACK = r"/mnt/data/gopro_006.txt"  # uploaded example
ANNOTATION_PATH = ANNOTATION_PATH_PRIMARY if os.path.exists(ANNOTATION_PATH_PRIMARY) else ANNOTATION_PATH_FALLBACK

# --- Output folder must be project-root/video_benchmark_outputs/ ---
OUTPUT_ROOT = "video_benchmark_outputs"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

# --- Inference policy ---
INFER_FPS = 5                     # inference steps per second (e.g., 5)
ROI_SIZE = 640                    # square crop size fed to YOLO
BASE_IMGSZ = 640                  # YOLO imgsz for baseline calls (can be raised, slower)
HIGHRES_IMGSZ = 1280              # used by VERIFY_MODE="DoublePassHighRes"
TILE_SIZE = 640                   # used by tiled inference modes
TILE_OVERLAP = 0.20               # 0.2 => 20% overlap
TILE_MAX_TILES = 64               # safety cap

MAX_FULLFRAME_DETECTIONS = 3
MAX_ROI_DETECTIONS = 1

# --- Motion ROI mining constraints ---
MIN_MOVER_W = 25
MIN_MOVER_H = 25
MAX_MOVER_ROIS = 3

# --- Confidence thresholds ---
BASE_CONF_FULL     = 0.25
GUIDED_ROI_CONF    = 0.40
LOW_CONF_DETECTION = 0.40

# --- Verification mode using SAME YOLO weights ---
# "None" | "DoublePassHighConf" | "DoublePassHighRes" | "DoublePassTiled"
VERIFY_MODE = "None"

# Verification / confirmation guided ROI
ENABLE_CONFIRMATION_GUIDED_ROI = True
REQUIRE_CONFIRMATION_GUIDED_ROI_FOR_ALL_DETECTIONS = False   # default OFF
FORCE_VERIFY_LOWCONF_OR_SMALL = True                         # rule #7

# --- Confirmation windows ---
# Separate windows:
# 1) CONFIRM (big objects only, unless verified)
# 2) WARNING (tracks everything, including small)
CONFIRM_WINDOW = 10
CONFIRM_REQUIRE = 9

WARNING_WINDOW = CONFIRM_WINDOW
WARNING_REQUIRE = CONFIRM_REQUIRE

CONFIRM_COOLDOWN_SEC = 1.0

# Size thresholds on ORIGINAL frame coordinates (not ROI coords)
MIN_CONFIRM_SIZE_BASE = 15        # rule #12
MIN_CONFIRM_SIZE_UNVERIFIED = 25  # rule #9
MIN_VERIFY_SIZE = 25              # rule #7
MIN_WARNING_SIZE = 1              # warning tracks anything detected

# --- Continuity gate selection ---
# "ExpandedIoU" | "CenterDistance" | "Kalman"
CONTINUITY_GATE = "ExpandedIoU"

# Expanded IoU gate parameters
EXPAND_FACTOR = 3.0               # expand previous box by this factor before IoU

# Center-distance gate parameters
CENTER_ALPHA = 2.5                # scales with sqrt(area)
CENTER_BETA  = 20.0               # extra pixels slack

# Kalman gate parameters (pixels)
KALMAN_GATE_PX = 120.0

SHOW_CONTINUITY_DEBUG = True
SHOW_GT_RECT = True

# --- Motion mining trigger ---
NO_DRONE_STREAK_FOR_MOTION = 5

# Motion buffering: number of inference frames stored for motion extraction
MOTION_BUFFER_STEPS = 5
MOTION_PERSIST_MIN_HITS = 3       # helps suppress leaves (must appear across frames)
MOTION_GLOBAL_RATIO_DISCARD = 0.40  # if motion mask covers >40% pixels, treat as global movement

# Motion method selection (object motion detector; not stackable with other motion methods)
# "None" | "MOG2" | "KNN" | "RunningAverage" | "TemporalMedian" | "FrameDifference"
# "ORB_Affine_StabilizedDiff" | "ECC_Affine_StabilizedDiff" | "OpticalFlowMagnitude" | "KLT_PointCluster"
PRE_MOTION_METHOD = "None"

# Guided ROI for next inference (tracking)
USE_GUIDED_ROI_NEXT_INFER = True

# Motion ROI mining toggle (will only run if PRE_MOTION_METHOD != "None")
USE_MOTION_MINING = True

# Tiny-object rescue mode (tiled inference) toggle
ENABLE_TINY_RESCUE_TILED = True
TINY_RESCUE_MAX_BOX = 25          # if best detected box <= 40x40, optionally try tiled verification

# --- Optional exports ---
SAVE_VIDEO = True
SHOW_PREVIEW = False
EXPORT_MOTION_DEBUG_IMAGES = True
EXPORT_TILED_DEBUG_IMAGES  = True
MAX_EXPORT_DEBUG_IMAGES = 300      # safety cap for each debug type

# Annotation label is exactly "drone"
DRONE_LABEL = "drone"

print("MODEL_WEIGHTS:", MODEL_WEIGHTS)
print("VIDEO_PATH:", VIDEO_PATH)
print("ANNOTATION_PATH:", ANNOTATION_PATH)
print("OUTPUT_ROOT:", os.path.abspath(OUTPUT_ROOT))

MODEL_WEIGHTS: S:\IntelliJ\Projects\ES_Drone_Detection\runs\detect\yolo26\drone_finetune_full_mixed\weights\best.pt
VIDEO_PATH: S:\IntelliJ\Projects\ES_Drone_Detection\video_test\gopro_006.mp4
ANNOTATION_PATH: S:\IntelliJ\Projects\ES_Drone_Detection\video_test\gopro_006.txt
OUTPUT_ROOT: S:\IntelliJ\Projects\ES_Drone_Detection\video_benchmark_outputs


## 3) Utilities
Cropping, IoU, drawing helpers, safe averaging, and simple NMS for merging tiled detections.

In [5]:
def safe_mean(xs):
    xs = [x for x in xs if x is not None]
    return float(sum(xs)/len(xs)) if xs else 0.0

def clamp(v, lo, hi):
    return max(lo, min(hi, v))

def xywh_to_xyxy(x, y, w, h):
    return (x, y, x + w, y + h)

def xyxy_to_xywh(x1, y1, x2, y2):
    return (x1, y1, x2 - x1, y2 - y1)

def box_area_xyxy(b):
    x1,y1,x2,y2 = b
    return max(0, x2-x1)*max(0, y2-y1)

def iou_xyxy(a, b):
    ax1,ay1,ax2,ay2 = a
    bx1,by1,bx2,by2 = b
    ix1,iy1 = max(ax1,bx1), max(ay1,by1)
    ix2,iy2 = min(ax2,bx2), min(ay2,by2)
    iw,ih = max(0, ix2-ix1), max(0, iy2-iy1)
    inter = iw*ih
    ua = box_area_xyxy(a) + box_area_xyxy(b) - inter
    return inter/ua if ua > 0 else 0.0

def center_of(b):
    x1,y1,x2,y2=b
    return ((x1+x2)/2.0, (y1+y2)/2.0)

def expand_box(b, factor, W, H):
    x1,y1,x2,y2=b
    cx,cy = center_of(b)
    w = (x2-x1)*factor
    h = (y2-y1)*factor
    nx1 = clamp(int(cx - w/2), 0, W-1)
    ny1 = clamp(int(cy - h/2), 0, H-1)
    nx2 = clamp(int(cx + w/2), 0, W-1)
    ny2 = clamp(int(cy + h/2), 0, H-1)
    if nx2 <= nx1: nx2 = min(W-1, nx1+1)
    if ny2 <= ny1: ny2 = min(H-1, ny1+1)
    return (nx1,ny1,nx2,ny2)

def crop_square_center(frame, cx, cy, size):
    H,W = frame.shape[:2]
    half = size // 2
    x1 = clamp(int(cx - half), 0, W - 1)
    y1 = clamp(int(cy - half), 0, H - 1)
    x2 = clamp(int(x1 + size), 1, W)
    y2 = clamp(int(y1 + size), 1, H)
    # if near border, shift back to keep square
    if x2 - x1 < size:
        x1 = clamp(x2 - size, 0, W - size)
        x2 = x1 + size
    if y2 - y1 < size:
        y1 = clamp(y2 - size, 0, H - size)
        y2 = y1 + size
    x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
    crop = frame[y1:y2, x1:x2].copy()
    return crop, (x1,y1,x2,y2)

def draw_box(img, b, color, thickness=2, label=None):
    x1,y1,x2,y2 = map(int,b)
    cv2.rectangle(img, (x1,y1), (x2,y2), color, thickness)
    if label:
        cv2.putText(img, label, (x1, max(0,y1-6)), cv2.FONT_HERSHEY_SIMPLEX, 0.55, color, 2, cv2.LINE_AA)

def nms_xyxy(boxes, scores, iou_thr=0.6):
    if not boxes:
        return []
    idxs = np.argsort(scores)[::-1]
    keep=[]
    while len(idxs)>0:
        i = idxs[0]
        keep.append(i)
        rest = []
        for j in idxs[1:]:
            if iou_xyxy(boxes[i], boxes[j]) < iou_thr:
                rest.append(j)
        idxs = np.array(rest, dtype=int)
    return keep

## 4) Annotation parser (your format)
Your `.txt` file format is **one line per frame**:
- `frame_id 0` → no drone
- `frame_id 1 x y w h drone` → one drone box


In [6]:
def load_annotations(path: str):
    gt = {}
    if not path or not os.path.exists(path):
        print("Annotation file not found:", path)
        return gt
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line=line.strip()
            if not line:
                continue
            parts=line.split()
            # expected: frame_id 0
            # or: frame_id 1 x y w h drone
            try:
                frame_id = int(parts[0])
            except:
                continue
            if len(parts) >= 2 and parts[1] == "0":
                gt[frame_id] = None
            elif len(parts) >= 7:
                # frame_id 1 x y w h label
                x = float(parts[2]); y=float(parts[3]); w=float(parts[4]); h=float(parts[5])
                label = parts[6].strip()
                if label == DRONE_LABEL:
                    gt[frame_id] = xywh_to_xyxy(x,y,w,h)
                else:
                    gt[frame_id] = None
            else:
                gt[frame_id] = None
    return gt

GT = load_annotations(ANNOTATION_PATH)
print("GT frames loaded:", len(GT))

GT frames loaded: 1351


## 5) Continuity gates (choose in config)
These gates decide whether a detection belongs to the **same target** across inference frames.
- **ExpandedIoU**: expands previous bbox (2–3×) before IoU. Good for low FPS.
- **CenterDistance**: compares centers with a size-scaled radius.
- **Kalman**: predicts next center using a constant-velocity Kalman filter, then gates by distance.


In [7]:
class ExpandedIoUGate:
    def __init__(self, expand_factor=3.0, iou_thr=0.05):
        self.expand_factor = expand_factor
        self.iou_thr = iou_thr
    def pass_gate(self, prev_box, curr_box, W, H):
        prev_exp = expand_box(prev_box, self.expand_factor, W, H)
        return iou_xyxy(prev_exp, curr_box) >= self.iou_thr, {"prev_exp": prev_exp}

class CenterDistanceGate:
    def __init__(self, alpha=2.5, beta=20.0):
        self.alpha=alpha
        self.beta=beta
    def pass_gate(self, prev_box, curr_box, W, H):
        pc = center_of(prev_box)
        cc = center_of(curr_box)
        dist = math.hypot(pc[0]-cc[0], pc[1]-cc[1])
        area = max(1.0, box_area_xyxy(prev_box))
        thr = self.alpha*math.sqrt(area) + self.beta
        return dist <= thr, {"pc": pc, "thr": thr}

class KalmanGate:
    # 2D constant velocity: state [x,y,vx,vy]
    def __init__(self, gate_px=120.0):
        self.gate_px = gate_px
        self.x = None
        self.P = None
        self.last_t = None
        # fixed noise
        self.Q_base = 5.0
        self.R = np.diag([25.0, 25.0]).astype(np.float32)
    def reset(self):
        self.x=None; self.P=None; self.last_t=None
    def predict(self, dt):
        F = np.array([[1,0,dt,0],
                      [0,1,0,dt],
                      [0,0,1,0],
                      [0,0,0,1]], dtype=np.float32)
        q = self.Q_base
        Q = np.diag([q,q,q,q]).astype(np.float32)
        self.x = F @ self.x
        self.P = F @ self.P @ F.T + Q
        return self.x.copy()
    def update(self, z):
        # z: [x,y]
        H = np.array([[1,0,0,0],
                      [0,1,0,0]], dtype=np.float32)
        y = z - (H @ self.x)
        S = H @ self.P @ H.T + self.R
        K = self.P @ H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        I = np.eye(4, dtype=np.float32)
        self.P = (I - K @ H) @ self.P
    def pass_gate(self, prev_box, curr_box, W, H, t=None):
        # Initialize on first call
        cc = np.array(center_of(curr_box), dtype=np.float32)
        if self.x is None:
            self.x = np.array([cc[0], cc[1], 0.0, 0.0], dtype=np.float32)
            self.P = np.eye(4, dtype=np.float32)*100.0
            self.last_t = t
            return True, {"pred": (cc[0], cc[1]), "gate": self.gate_px}
        dt = 1.0 if self.last_t is None or t is None else max(1e-3, float(t - self.last_t))
        pred = self.predict(dt)
        pred_xy = (float(pred[0]), float(pred[1]))
        dist = math.hypot(pred_xy[0]-cc[0], pred_xy[1]-cc[1])
        ok = dist <= self.gate_px
        # update if accepted, else keep prediction (do not latch onto noise)
        if ok:
            self.update(cc)
            self.last_t = t
        return ok, {"pred": pred_xy, "gate": self.gate_px}

def make_gate():
    if CONTINUITY_GATE == "ExpandedIoU":
        return ExpandedIoUGate(expand_factor=EXPAND_FACTOR, iou_thr=CONFIRM_IOU_CONTINUITY if 'CONFIRM_IOU_CONTINUITY' in globals() else 0.05)
    if CONTINUITY_GATE == "CenterDistance":
        return CenterDistanceGate(alpha=CENTER_ALPHA, beta=CENTER_BETA)
    if CONTINUITY_GATE == "Kalman":
        return KalmanGate(gate_px=KALMAN_GATE_PX)
    return ExpandedIoUGate(expand_factor=EXPAND_FACTOR, iou_thr=0.05)

In [8]:
# IoU threshold used by ExpandedIoU gate after expansion
CONFIRM_IOU_CONTINUITY = 0.05

## 6) Motion detectors (select one)
Motion ROI mining runs only after `NO_DRONE_STREAK_FOR_MOTION` consecutive inference steps without an accepted drone.
It outputs **1 to 3** ROIs and runs YOLO on each ROI (max 3 YOLO calls per inference step).

Leaf/trees suppression used here:
- temporal persistence across buffered inference frames (`MOTION_PERSIST_MIN_HITS`)
- discard if the motion mask covers too much of the frame (`MOTION_GLOBAL_RATIO_DISCARD`) → treat as global movement


In [9]:
def to_gray_blur(frame):
    g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    g = cv2.GaussianBlur(g, (5,5), 0)
    return g

def motion_mask_framediff(prev_g, curr_g, thr=20):
    d = cv2.absdiff(prev_g, curr_g)
    _, m = cv2.threshold(d, thr, 255, cv2.THRESH_BINARY)
    return m

def motion_mask_optflow(prev_g, curr_g, mag_thr=1.5):
    flow = cv2.calcOpticalFlowFarneback(prev_g, curr_g, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, _ = cv2.cartToPolar(flow[...,0], flow[...,1])
    m = (mag > mag_thr).astype(np.uint8)*255
    return m

def estimate_affine_orb(prev_g, curr_g):
    orb = cv2.ORB_create(500)
    kp1, des1 = orb.detectAndCompute(prev_g, None)
    kp2, des2 = orb.detectAndCompute(curr_g, None)
    if des1 is None or des2 is None or len(kp1) < 10 or len(kp2) < 10:
        return None
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    if len(matches) < 10:
        return None
    matches = sorted(matches, key=lambda x: x.distance)[:80]
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1,1,2)
    M, _ = cv2.estimateAffinePartial2D(pts1, pts2, method=cv2.RANSAC, ransacReprojThreshold=3.0)
    return M

def motion_mask_orb_stabilized(prev_g, curr_g, thr=20):
    H,W = curr_g.shape[:2]
    M = estimate_affine_orb(prev_g, curr_g)
    if M is None:
        return motion_mask_framediff(prev_g, curr_g, thr=thr)
    warped = cv2.warpAffine(prev_g, M, (W,H), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
    return motion_mask_framediff(warped, curr_g, thr=thr)

def motion_mask_ecc_stabilized(prev_g, curr_g, thr=20):
    H,W = curr_g.shape[:2]
    warp = np.eye(2,3, dtype=np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 50, 1e-4)
    try:
        cc, warp = cv2.findTransformECC(prev_g, curr_g, warp, cv2.MOTION_AFFINE, criteria)
        warped = cv2.warpAffine(prev_g, warp, (W,H), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
        return motion_mask_framediff(warped, curr_g, thr=thr)
    except:
        return motion_mask_framediff(prev_g, curr_g, thr=thr)

def klt_clusters(prev_g, curr_g, max_pts=200):
    p0 = cv2.goodFeaturesToTrack(prev_g, maxCorners=max_pts, qualityLevel=0.01, minDistance=7, blockSize=7)
    if p0 is None:
        return np.zeros_like(prev_g, dtype=np.uint8)
    p1, st, err = cv2.calcOpticalFlowPyrLK(prev_g, curr_g, p0, None, winSize=(21,21), maxLevel=3,
                                           criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    if p1 is None:
        return np.zeros_like(prev_g, dtype=np.uint8)
    good0 = p0[st==1]
    good1 = p1[st==1]
    if len(good0) < 10:
        return np.zeros_like(prev_g, dtype=np.uint8)
    # global motion
    flow = good1 - good0
    med = np.median(flow, axis=0)
    res = flow - med
    mag = np.linalg.norm(res, axis=1)
    # keep points with residual motion
    keep = mag > 2.0
    mask = np.zeros_like(prev_g, dtype=np.uint8)
    for (x,y) in good1[keep]:
        cv2.circle(mask, (int(x),int(y)), 5, 255, -1)
    return mask

class MotionMiner:
    def __init__(self, method: str):
        self.method = method
        self.mog2 = None
        self.knn = None
        self.runavg = None
        self.runavg_alpha = 0.02
        self.temporal = deque(maxlen=max(5, MOTION_BUFFER_STEPS))
    def _init_bg(self):
        if self.method == "MOG2" and self.mog2 is None:
            self.mog2 = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)
        if self.method == "KNN" and self.knn is None:
            self.knn = cv2.createBackgroundSubtractorKNN(history=500, dist2Threshold=400.0, detectShadows=True)
    def mask_from_pair(self, prev_g, curr_g):
        if self.method == "FrameDifference":
            return motion_mask_framediff(prev_g, curr_g, thr=20)
        if self.method == "ORB_Affine_StabilizedDiff":
            return motion_mask_orb_stabilized(prev_g, curr_g, thr=20)
        if self.method == "ECC_Affine_StabilizedDiff":
            return motion_mask_ecc_stabilized(prev_g, curr_g, thr=20)
        if self.method == "OpticalFlowMagnitude":
            return motion_mask_optflow(prev_g, curr_g, mag_thr=1.5)
        if self.method == "KLT_PointCluster":
            return klt_clusters(prev_g, curr_g)
        return motion_mask_framediff(prev_g, curr_g, thr=20)
    def update_and_get_mask(self, frame_bgr):
        g = to_gray_blur(frame_bgr)
        self.temporal.append(g)
        self._init_bg()
        if self.method in ("MOG2","KNN"):
            if self.method == "MOG2":
                fg = self.mog2.apply(frame_bgr)
            else:
                fg = self.knn.apply(frame_bgr)
            # clean noise
            k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
            fg = cv2.morphologyEx(fg, cv2.MORPH_OPEN, k)
            return fg
        if self.method == "RunningAverage":
            if self.runavg is None:
                self.runavg = g.astype(np.float32)
                return np.zeros_like(g, dtype=np.uint8)
            cv2.accumulateWeighted(g.astype(np.float32), self.runavg, self.runavg_alpha)
            bg = cv2.convertScaleAbs(self.runavg)
            return motion_mask_framediff(bg, g, thr=20)
        if self.method == "TemporalMedian":
            if len(self.temporal) < self.temporal.maxlen:
                return np.zeros_like(g, dtype=np.uint8)
            stack = np.stack(list(self.temporal), axis=0)
            med = np.median(stack, axis=0).astype(np.uint8)
            return motion_mask_framediff(med, g, thr=20)
        # pair-based methods need prev frame
        if len(self.temporal) < 2:
            return np.zeros_like(g, dtype=np.uint8)
        prev = self.temporal[-2]
        curr = self.temporal[-1]
        return self.mask_from_pair(prev, curr)

def rois_from_mask(mask, W, H, min_w, min_h, max_rois):
    # morphology to connect
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    m = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k)
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN, k)
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    rois=[]
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        if w < min_w or h < min_h:
            continue
        rois.append((x,y,x+w,y+h, w*h))
    rois.sort(key=lambda t: t[4], reverse=True)
    return [(x1,y1,x2,y2) for (x1,y1,x2,y2,_) in rois[:max_rois]], m

## 7) YOLO wrappers + tiled inference
Tiled inference increases effective resolution for small objects by running YOLO over overlapping tiles and merging detections.


In [10]:
def load_model(weights):
    model = YOLO(weights)
    return model

model = load_model(MODEL_WEIGHTS)
names = model.model.names if hasattr(model, "model") and hasattr(model.model, "names") else {}
print("Model names:", names)

# Find class id for "drone" by exact label match preference
def find_drone_class_id(names_dict):
    if isinstance(names_dict, dict):
        for k,v in names_dict.items():
            if str(v).strip().lower() == DRONE_LABEL:
                return int(k)
    # fallback: if only 1 class, assume it is drone
    if isinstance(names_dict, dict) and len(names_dict)==1:
        return int(list(names_dict.keys())[0])
    return None

DRONE_CLASS_ID = find_drone_class_id(names)
print("DRONE_CLASS_ID:", DRONE_CLASS_ID)

def run_yolo_on_image(img_bgr, conf, max_det, imgsz):
    t0 = time.perf_counter()
    res = model.predict(img_bgr, conf=conf, max_det=max_det, imgsz=imgsz, verbose=False)[0]
    infer_ms = (time.perf_counter()-t0)*1000.0
    boxes=[]
    confs=[]
    clss=[]
    if res.boxes is not None and len(res.boxes) > 0:
        b = res.boxes.xyxy.cpu().numpy()
        c = res.boxes.conf.cpu().numpy()
        k = res.boxes.cls.cpu().numpy()
        for i in range(len(b)):
            boxes.append(tuple(map(float, b[i])))
            confs.append(float(c[i]))
            clss.append(int(k[i]))
    return boxes, confs, clss, infer_ms

def filter_drone(boxes, confs, clss):
    out=[]
    for b,cf,cl in zip(boxes, confs, clss):
        if DRONE_CLASS_ID is None or cl == DRONE_CLASS_ID:
            out.append((b, cf))
    out.sort(key=lambda t: t[1], reverse=True)
    return out

def tiled_inference(frame_bgr, conf, max_det, imgsz, tile_size=640, overlap=0.2, max_tiles=64):
    H,W = frame_bgr.shape[:2]
    step = int(tile_size * (1.0 - overlap))
    step = max(1, step)
    boxes_all=[]
    confs_all=[]
    tiles=[]
    for y in range(0, H, step):
        for x in range(0, W, step):
            x2 = min(W, x+tile_size)
            y2 = min(H, y+tile_size)
            x1 = max(0, x2 - tile_size)
            y1 = max(0, y2 - tile_size)
            tiles.append((x1,y1,x2,y2))
            if len(tiles) >= max_tiles:
                break
        if len(tiles) >= max_tiles:
            break
    infer_ms_total=0.0
    for (x1,y1,x2,y2) in tiles:
        tile = frame_bgr[y1:y2, x1:x2]
        b, c, k, ms = run_yolo_on_image(tile, conf=conf, max_det=max_det, imgsz=imgsz)
        infer_ms_total += ms
        dets = filter_drone(b,c,k)
        for (bb, cf) in dets:
            bx1,by1,bx2,by2 = bb
            boxes_all.append((bx1+x1, by1+y1, bx2+x1, by2+y1))
            confs_all.append(cf)
    if not boxes_all:
        return [], [], infer_ms_total, tiles
    keep = nms_xyxy(boxes_all, confs_all, iou_thr=0.6)
    boxes_n = [boxes_all[i] for i in keep]
    confs_n = [confs_all[i] for i in keep]
    # sort by conf
    order = np.argsort(confs_n)[::-1]
    boxes_n = [boxes_n[i] for i in order]
    confs_n = [confs_n[i] for i in order]
    return boxes_n, confs_n, infer_ms_total, tiles

Model names: {0: 'drone'}
DRONE_CLASS_ID: 0


## 8) Main pipeline
Rules implemented:
- Overlay updates on inference frames, but is drawn on **all frames** (no flicker).
- Motion mining: after `NO_DRONE_STREAK_FOR_MOTION`, run 1–3 ROI inferences and draw all yellow ROIs.
- Guided ROI (cyan) is used for the **next inference** after an accepted detection.
- Confirmation-guided ROI (magenta) is optional post-pass on the **same inference**.
- Continuity gate is selectable and can be visualized.
- If motion indicates global scene movement, motion ROIs are discarded (full-frame used).
- One inference step = one hit max (even if multiple ROI inferences run).


In [11]:
@dataclass
class StepDecision:
    has_drone: bool
    primary_box: Optional[Tuple[float,float,float,float]]
    primary_conf: float
    source: str                      # "FullFrame" | "GuidedROI" | "MotionROI" | "Tiled" | "Verify"
    yolo_calls: int
    boxes_all: List[Tuple[float,float,float,float]]  # all drone boxes (orig coords)
    confs_all: List[float]
    verified: bool
    verify_source: str

def box_w_h(b):
    x1,y1,x2,y2=b
    return max(0,x2-x1), max(0,y2-y1)

def is_small_box(b, min_side):
    w,h = box_w_h(b)
    return (w < min_side) or (h < min_side)

def size_ok_for_confirm(b, verified):
    w,h = box_w_h(b)
    if w < MIN_CONFIRM_SIZE_BASE or h < MIN_CONFIRM_SIZE_BASE:
        return False
    if (w < MIN_CONFIRM_SIZE_UNVERIFIED or h < MIN_CONFIRM_SIZE_UNVERIFIED) and (not verified):
        return False
    return True

def make_run_id(video_path):
    base = os.path.splitext(os.path.basename(video_path))[0]
    return f"{base}__infer{INFER_FPS}fps__motion-{PRE_MOTION_METHOD}__verify-{VERIFY_MODE}"

RUN_ID = make_run_id(VIDEO_PATH)
OUT_VIDEO_PATH = os.path.join(OUTPUT_ROOT, f"{RUN_ID}.mp4")
PER_FRAME_CSV  = os.path.join(OUTPUT_ROOT, f"{RUN_ID}__per_frame_log.csv")
MASTER_SUMMARY_CSV = os.path.join(OUTPUT_ROOT, "benchmark_runs_summary.csv")

MOTION_DEBUG_DIR = os.path.join(OUTPUT_ROOT, "motion_debug")
TILED_DEBUG_DIR  = os.path.join(OUTPUT_ROOT, "tiled_debug")
os.makedirs(MOTION_DEBUG_DIR, exist_ok=True)
os.makedirs(TILED_DEBUG_DIR, exist_ok=True)

def decide_inference_inputs(frame_bgr, W, H, forced_guided_rect, do_motion, motion_rois):
    inputs=[]
    # guided ROI for NEXT inference (cyan), if available
    if forced_guided_rect is not None:
        x1,y1,x2,y2 = forced_guided_rect
        crop = frame_bgr[y1:y2, x1:x2]
        inputs.append(("GuidedROI", crop, forced_guided_rect))
        return inputs  # guided ROI ignores motion/baseline in that inference step

    if do_motion and motion_rois:
        for r in motion_rois:
            x1,y1,x2,y2 = r
            cx,cy = (x1+x2)/2.0, (y1+y2)/2.0
            crop, rect = crop_square_center(frame_bgr, cx, cy, ROI_SIZE)
            inputs.append(("MotionROI", crop, rect))
        return inputs

    # baseline full frame
    inputs.append(("FullFrame", frame_bgr, (0,0,W,H)))
    return inputs

def verification_pass(frame_bgr, primary_box, mode):
    # returns: (verified_bool, verified_box, verified_conf, verify_calls, verify_ms, debug_tiles)
    H,W = frame_bgr.shape[:2]
    cx,cy = center_of(primary_box)
    debug_tiles=None

    if mode == "DoublePassHighConf":
        crop, rect = crop_square_center(frame_bgr, cx, cy, ROI_SIZE)
        b,c,k,ms = run_yolo_on_image(crop, conf=GUIDED_ROI_CONF, max_det=MAX_ROI_DETECTIONS, imgsz=BASE_IMGSZ)
        dets = filter_drone(b,c,k)
        if dets:
            vb, vconf = dets[0]
            # map to orig
            x1,y1,x2,y2 = rect
            vb_o = (vb[0]+x1, vb[1]+y1, vb[2]+x1, vb[3]+y1)
            return True, vb_o, vconf, 1, ms, debug_tiles
        return False, None, 0.0, 1, ms, debug_tiles

    if mode == "DoublePassHighRes":
        # larger crop + higher imgsz
        big = min(max(W,H), ROI_SIZE*2)
        crop, rect = crop_square_center(frame_bgr, cx, cy, big)
        b,c,k,ms = run_yolo_on_image(crop, conf=GUIDED_ROI_CONF, max_det=MAX_ROI_DETECTIONS, imgsz=HIGHRES_IMGSZ)
        dets = filter_drone(b,c,k)
        if dets:
            vb, vconf = dets[0]
            x1,y1,x2,y2 = rect
            vb_o = (vb[0]+x1, vb[1]+y1, vb[2]+x1, vb[3]+y1)
            return True, vb_o, vconf, 1, ms, debug_tiles
        return False, None, 0.0, 1, ms, debug_tiles

    if mode == "DoublePassTiled":
        big = min(max(W,H), ROI_SIZE*2)
        crop, rect = crop_square_center(frame_bgr, cx, cy, big)
        boxes, confs, ms, tiles = tiled_inference(crop, conf=GUIDED_ROI_CONF, max_det=MAX_ROI_DETECTIONS, imgsz=BASE_IMGSZ,
                                                  tile_size=TILE_SIZE, overlap=TILE_OVERLAP, max_tiles=TILE_MAX_TILES)
        debug_tiles = [(tx+rect[0], ty+rect[1], tx2+rect[0], ty2+rect[1]) for (tx,ty,tx2,ty2) in tiles]
        if boxes:
            vb_o = (boxes[0][0]+rect[0], boxes[0][1]+rect[1], boxes[0][2]+rect[0], boxes[0][3]+rect[1])
            return True, vb_o, float(confs[0]), len(tiles), ms, debug_tiles
        return False, None, 0.0, len(tiles), ms, debug_tiles

    return False, None, 0.0, 0, 0.0, debug_tiles

def build_motion_rois(motion_masks, W, H):
    # temporal persistence to reduce leaves
    if not motion_masks:
        return [], None, 0.0
    acc = np.zeros_like(motion_masks[0], dtype=np.uint16)
    for m in motion_masks:
        acc += (m > 0).astype(np.uint16)
    # keep pixels that moved in at least MOTION_PERSIST_MIN_HITS masks
    keep = (acc >= MOTION_PERSIST_MIN_HITS).astype(np.uint8)*255

    motion_ratio = float(np.count_nonzero(keep)) / float(W*H)
    return keep, acc, motion_ratio

def ensure_master_summary_header(path):
    if os.path.exists(path):
        return
    cols = [
        "run_id","video","infer_fps","pre_motion_method","use_motion","use_guided_next","verify_mode",
        "confirm_window","confirm_require","warning_window","warning_require",
        "base_conf_full","guided_roi_conf","low_conf_detection",
        "precision_infer","recall_infer","ap50_infer",
        "avg_conf_infer",
        "avg_pre_ms","avg_infer_ms","avg_post_ms",
        "confirmed_events","warning_events",
        "notes"
    ]
    with open(path,"w",newline="",encoding="utf-8") as f:
        w=csv.writer(f); w.writerow(cols)

ensure_master_summary_header(MASTER_SUMMARY_CSV)

## 9) Run benchmark
This cell:
- Reads the video
- Runs inference at `INFER_FPS`
- Exports output video + per-frame CSV
- Appends a row to `video_benchmark_outputs/benchmark_runs_summary.csv`


In [12]:
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f"Cannot open video: {VIDEO_PATH}")

fps = cap.get(cv2.CAP_PROP_FPS)
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

infer_stride = max(1, int(round(fps / float(INFER_FPS))))
print("Video FPS:", fps, "Frames:", frame_count, "Infer stride:", infer_stride)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = None
if SAVE_VIDEO:
    writer = cv2.VideoWriter(OUT_VIDEO_PATH, fourcc, fps, (W,H))

# State
motion_miner = MotionMiner(PRE_MOTION_METHOD) if PRE_MOTION_METHOD != "None" else None
motion_masks_buffer = deque(maxlen=MOTION_BUFFER_STEPS)
infer_frames_gray_buffer = deque(maxlen=MOTION_BUFFER_STEPS)

forced_guided_rect_next = None   # cyan (next inference)
no_drone_streak = 0

gate = make_gate()
kalman_gate = gate if isinstance(gate, KalmanGate) else None

# confirmation windows
confirm_hits = deque(maxlen=CONFIRM_WINDOW)    # entries: (hit_bool, box, verified, t)
warning_hits = deque(maxlen=WARNING_WINDOW)

last_confirm_time = -1e9
last_warning_time = -1e9
confirmed_events = 0
warning_events = 0

# Counters
counts = defaultdict(int)
pre_ms_list=[]
infer_ms_list=[]
post_ms_list=[]
avg_conf_list=[]

# For AP50 on inference frames
infer_eval = []  # list of dicts: {frame_id, pred_box, pred_conf, gt_box}

# overlay state persists across non-infer frames (no flicker)
overlay_state = {
    "last_infer_frame": -1,
    "last_infer_time_s": 0.0,
    "last_mode": "Baseline",
    "last_source": "FullFrame",
    "last_has_drone": False,
    "last_conf": 0.0,
    "last_box": None,
    "last_verified": False,
    "last_verify_mode": "None",
    "last_yolo_calls": 0,
    "motion_rois_used": 0,
    "motion_rois_rects": [],
    "guided_rect_used": None,
    "confirm_guided_rect": None,
    "continuity_debug": None,
    "tiles_debug": None,
}

# Debug export counters
motion_dbg_saved = 0
tiled_dbg_saved  = 0

# per-frame csv
csv_cols = [
    "frame_id","time_s","is_infer_frame",
    "mode","source","has_drone","primary_conf","primary_box",
    "verified","verify_mode","yolo_calls",
    "gt_box",
    "confirm_state","warning_state",
    "confirmed_events","warning_events",
    "no_drone_streak",
    "count_full_hits","count_guided_hits","count_motion_hits","count_verified_hits"
]
csv_rows=[]

def overlay_lines(frame_id, time_s):
    # Use overlay_state values, updated on inference frames only
    lines=[]
    lines.append(f"Frame: {frame_id}  Time: {time_s:.2f}s")
    lines.append(f"Inference frame: {'YES' if (frame_id % infer_stride == 0) else 'NO'}  (infer_fps={INFER_FPS}, stride={infer_stride})")
    lines.append(f"Mode: {overlay_state['last_mode']} | Source: {overlay_state['last_source']} | YOLO calls (this inference): {overlay_state['last_yolo_calls']}")
    lines.append(f"Drone detected: {overlay_state['last_has_drone']} | Conf: {overlay_state['last_conf']:.3f} | Verified: {overlay_state['last_verified']} ({overlay_state['last_verify_mode']})")
    lines.append(f"Hits so far: FullFrame={counts['full_hit']} GuidedNext={counts['guided_hit']} MotionROI={counts['motion_hit']} Verified={counts['verified_hit']}")
    lines.append(f"Confirmed events: {confirmed_events} | Warning events: {warning_events} | No-drone streak: {no_drone_streak}")
    return lines

def compute_ap50(infer_eval_rows):
    # One GT per frame at most. Compute AP50 like detection AP:
    # sort predictions by conf, TP if IoU>=0.5 with GT on that frame, else FP.
    # frames without GT are ignored for recall denominator but included for precision via FP.
    preds=[]
    gt_frames=set()
    for r in infer_eval_rows:
        if r["gt_box"] is not None:
            gt_frames.add(r["frame_id"])
        if r["pred_box"] is not None:
            preds.append((r["pred_conf"], r["frame_id"], r["pred_box"], r["gt_box"]))
    preds.sort(key=lambda t: t[0], reverse=True)
    matched=set()
    tps=[]
    fps=[]
    tp=0
    fp=0
    for conf, fid, pb, gb in preds:
        if gb is not None and fid not in matched and iou_xyxy(pb, gb) >= 0.5:
            tp += 1
            matched.add(fid)
            tps.append(tp); fps.append(fp)
        else:
            fp += 1
            tps.append(tp); fps.append(fp)
    if not gt_frames:
        return 0.0
    precisions=[]
    recalls=[]
    for i in range(len(preds)):
        p = tps[i] / max(1, (tps[i]+fps[i]))
        r = tps[i] / len(gt_frames)
        precisions.append(p); recalls.append(r)
    # 11-point interp AP
    ap=0.0
    for t in np.linspace(0,1,11):
        pmax = 0.0
        for p,r in zip(precisions, recalls):
            if r >= t:
                pmax = max(pmax, p)
        ap += pmax
    ap /= 11.0
    return float(ap)

def update_window(window_deque, hit_ok, box, verified, t, W, H):
    # continuity: if previous hit exists, gate must pass; else accept
    dbg=None
    if hit_ok and box is not None:
        prev = None
        for e in reversed(window_deque):
            if e[0] and e[1] is not None:
                prev = e[1]
                break
        if prev is not None:
            if isinstance(gate, KalmanGate):
                ok, info = gate.pass_gate(prev, box, W, H, t=t)
                dbg=info
            else:
                ok, info = gate.pass_gate(prev, box, W, H)
                dbg=info
            if not ok:
                hit_ok = False
    window_deque.append((hit_ok, box, verified, t, dbg))
    return hit_ok, dbg

frame_id=0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    t_s = frame_id / fps

    is_infer = (frame_id % infer_stride == 0)

    # --- draw overlay later; we first maybe update overlay_state if infer ---
    if is_infer:
        pre_t0 = time.perf_counter()

        # Update motion buffer (for motion ROI mining)
        g = to_gray_blur(frame)
        infer_frames_gray_buffer.append(g)

        # Update motion miner mask buffer if enabled and method chosen
        current_motion_mask = None
        if motion_miner is not None:
            current_motion_mask = motion_miner.update_and_get_mask(frame)
            motion_masks_buffer.append(current_motion_mask)

        # decide whether to motion-mine this inference
        do_motion = (USE_MOTION_MINING and motion_miner is not None and no_drone_streak >= NO_DRONE_STREAK_FOR_MOTION)
        motion_rois=[]
        motion_keep_mask=None
        motion_ratio=0.0
        motion_acc=None

        if do_motion and len(motion_masks_buffer) >= 2:
            keep_mask, acc, ratio = build_motion_rois(list(motion_masks_buffer), W, H)
            motion_keep_mask = keep_mask
            motion_acc = acc
            motion_ratio = ratio
            if motion_ratio > MOTION_GLOBAL_RATIO_DISCARD:
                # global movement -> discard motion ROIs
                do_motion = False
            else:
                rois, cleaned = rois_from_mask(motion_keep_mask, W, H, MIN_MOVER_W, MIN_MOVER_H, MAX_MOVER_ROIS)
                motion_rois = rois

        # Prepare inputs (baseline/guided/motion)
        inputs = decide_inference_inputs(frame, W, H, forced_guided_rect_next if USE_GUIDED_ROI_NEXT_INFER else None, do_motion, motion_rois)

        forced_guided_rect_next = None  # consumed

        pre_ms = (time.perf_counter()-pre_t0)*1000.0

        # Run YOLO on selected inputs
        infer_calls = 0
        infer_ms = 0.0
        drone_boxes=[]
        drone_confs=[]
        primary_source="FullFrame"

        # For visualization on this inference frame
        overlay_state["motion_rois_rects"] = []
        overlay_state["guided_rect_used"] = None
        overlay_state["confirm_guided_rect"] = None
        overlay_state["continuity_debug"] = None
        overlay_state["tiles_debug"] = None

        for idx,(src, img, rect) in enumerate(inputs):
            if src == "FullFrame":
                conf = BASE_CONF_FULL
                max_det = MAX_FULLFRAME_DETECTIONS
                imgsz = BASE_IMGSZ
            elif src == "GuidedROI":
                conf = BASE_CONF_FULL  # first pass threshold; verification uses GUIDED_ROI_CONF
                max_det = MAX_ROI_DETECTIONS
                imgsz = BASE_IMGSZ
                overlay_state["guided_rect_used"] = rect
            elif src == "MotionROI":
                conf = BASE_CONF_FULL
                max_det = MAX_ROI_DETECTIONS
                imgsz = BASE_IMGSZ
                overlay_state["motion_rois_rects"].append(rect)
            else:
                conf = BASE_CONF_FULL; max_det = MAX_ROI_DETECTIONS; imgsz = BASE_IMGSZ

            b,c,k,ms = run_yolo_on_image(img, conf=conf, max_det=max_det, imgsz=imgsz)
            infer_calls += 1
            infer_ms += ms

            dets = filter_drone(b,c,k)
            if dets:
                # map to original coords
                rx1,ry1,rx2,ry2 = rect
                for (bb, cf) in dets:
                    bx1,by1,bx2,by2 = bb
                    drone_boxes.append((bx1+rx1, by1+ry1, bx2+rx1, by2+ry1))
                    drone_confs.append(cf)
                if primary_source == "FullFrame":
                    primary_source = src

        # pick primary
        has_drone = len(drone_boxes) > 0
        primary_box = None
        primary_conf = 0.0
        if has_drone:
            best_i = int(np.argmax(drone_confs))
            primary_box = drone_boxes[best_i]
            primary_conf = float(drone_confs[best_i])

        verified=False
        verify_calls=0
        verify_ms=0.0
        verify_source="None"
        verify_tiles=None

        # Decide if we must verify on same inference (confirmation guided ROI)
        must_verify = False
        if has_drone and primary_box is not None:
            if (primary_conf < LOW_CONF_DETECTION) or is_small_box(primary_box, MIN_VERIFY_SIZE):
                must_verify = True
        if REQUIRE_CONFIRMATION_GUIDED_ROI_FOR_ALL_DETECTIONS and has_drone:
            must_verify = True

        # Do verification if enabled/forced
        if has_drone and primary_box is not None and ( (ENABLE_CONFIRMATION_GUIDED_ROI and must_verify) or (FORCE_VERIFY_LOWCONF_OR_SMALL and must_verify) ):
            post_t0 = time.perf_counter()
            # If VERIFY_MODE=="None" but verification is mandatory, default to HighConf
            mode = VERIFY_MODE if VERIFY_MODE != "None" else "DoublePassHighConf"
            verified, vbox, vconf, vcalls, vms, vtiles = verification_pass(frame, primary_box, mode)
            verify_calls += vcalls
            verify_ms += vms
            verify_tiles = vtiles
            overlay_state["confirm_guided_rect"] = expand_box(primary_box, 2.0, W, H)
            overlay_state["tiles_debug"] = verify_tiles
            verify_source = mode

            if not verified:
                # dismiss original detection as false positive
                has_drone = False
                primary_box = None
                primary_conf = 0.0
                drone_boxes=[]
                drone_confs=[]
            else:
                # replace with verified box/conf if available
                if vbox is not None:
                    primary_box = vbox
                if vconf is not None:
                    primary_conf = float(vconf)
                counts["verified_hit"] += 1
            post_ms = (time.perf_counter()-post_t0)*1000.0
        else:
            post_ms = 0.0

        # Tiny rescue tiled (only when enabled and still no drone AND motion mining didn't trigger)
        if (not has_drone) and ENABLE_TINY_RESCUE_TILED and (no_drone_streak >= NO_DRONE_STREAK_FOR_MOTION) and (PRE_MOTION_METHOD == "None"):
            post_t0 = time.perf_counter()
            boxes_t, confs_t, ms_t, tiles = tiled_inference(frame, conf=BASE_CONF_FULL, max_det=MAX_FULLFRAME_DETECTIONS, imgsz=BASE_IMGSZ,
                                                            tile_size=TILE_SIZE, overlap=TILE_OVERLAP, max_tiles=TILE_MAX_TILES)
            infer_calls += len(tiles)
            infer_ms += ms_t
            if boxes_t:
                has_drone = True
                primary_box = boxes_t[0]
                primary_conf = float(confs_t[0])
                primary_source = "Tiled"
                overlay_state["tiles_debug"] = tiles
            post_ms += (time.perf_counter()-post_t0)*1000.0

        # Update guided ROI for next inference if enabled and detection accepted
        if has_drone and primary_box is not None and USE_GUIDED_ROI_NEXT_INFER:
            cx,cy = center_of(primary_box)
            _, rect = crop_square_center(frame, cx, cy, ROI_SIZE)
            forced_guided_rect_next = rect

        # Update streak
        if has_drone:
            no_drone_streak = 0
        else:
            no_drone_streak += 1

        # Update windows (warning + confirm)
        hit_t = t_s
        warn_hit_ok = has_drone and primary_box is not None
        warn_hit_ok, warn_dbg = update_window(warning_hits, warn_hit_ok, primary_box, verified, hit_t, W, H)

        conf_hit_ok = False
        if has_drone and primary_box is not None:
            conf_hit_ok = size_ok_for_confirm(primary_box, verified)
        conf_hit_ok, conf_dbg = update_window(confirm_hits, conf_hit_ok, primary_box, verified, hit_t, W, H)

        overlay_state["continuity_debug"] = conf_dbg if conf_dbg is not None else warn_dbg

        # event triggering
        def count_hits(window):
            return sum(1 for e in window if e[0])

        now = t_s
        if (now - last_confirm_time) >= CONFIRM_COOLDOWN_SEC:
            if count_hits(confirm_hits) >= CONFIRM_REQUIRE:
                confirmed_events += 1
                last_confirm_time = now

        if (now - last_warning_time) >= CONFIRM_COOLDOWN_SEC:
            if count_hits(warning_hits) >= WARNING_REQUIRE:
                warning_events += 1
                last_warning_time = now

        # Update hit counters by source
        if has_drone:
            if primary_source == "FullFrame": counts["full_hit"] += 1
            elif primary_source == "GuidedROI": counts["guided_hit"] += 1
            elif primary_source == "MotionROI": counts["motion_hit"] += 1
            elif primary_source == "Tiled": counts["tiled_hit"] += 1

        # Evaluate AP50 inputs (inference frames only)
        gt_box = GT.get(frame_id, None)
        infer_eval.append({
            "frame_id": frame_id,
            "pred_box": primary_box if has_drone else None,
            "pred_conf": primary_conf if has_drone else None,
            "gt_box": gt_box
        })

        # Update overlay_state for all non-infer frames to reuse
        overlay_state.update({
            "last_infer_frame": frame_id,
            "last_infer_time_s": t_s,
            "last_mode": "GuidedNext" if primary_source=="GuidedROI" else ("MotionMining" if do_motion and motion_rois else "Baseline"),
            "last_source": primary_source,
            "last_has_drone": bool(has_drone),
            "last_conf": float(primary_conf),
            "last_box": primary_box,
            "last_verified": bool(verified),
            "last_verify_mode": verify_source,
            "last_yolo_calls": int(infer_calls),
            "motion_rois_used": len(motion_rois) if do_motion else 0,
        })

        pre_ms_list.append(pre_ms)
        infer_ms_list.append(infer_ms)
        post_ms_list.append(post_ms)
        if has_drone:
            avg_conf_list.append(primary_conf)

        # Debug exports
        if EXPORT_MOTION_DEBUG_IMAGES and do_motion and motion_keep_mask is not None and motion_dbg_saved < MAX_EXPORT_DEBUG_IMAGES:
            dbg = frame.copy()
            # draw all motion ROIs (yellow)
            for r in motion_rois:
                draw_box(dbg, r, (0,255,255), 2, "Motion ROI (mask bbox)")
            # add ratio
            cv2.putText(dbg, f"motion_ratio={motion_ratio:.3f}", (10, H-20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2, cv2.LINE_AA)
            outp = os.path.join(MOTION_DEBUG_DIR, f"{RUN_ID}__infer{frame_id:06d}.jpg")
            cv2.imwrite(outp, dbg)
            motion_dbg_saved += 1

        if EXPORT_TILED_DEBUG_IMAGES and overlay_state["tiles_debug"] is not None and tiled_dbg_saved < MAX_EXPORT_DEBUG_IMAGES:
            dbg = frame.copy()
            for r in overlay_state["tiles_debug"]:
                draw_box(dbg, r, (255,255,0), 1, None)
            outp = os.path.join(TILED_DEBUG_DIR, f"{RUN_ID}__infer{frame_id:06d}.jpg")
            cv2.imwrite(outp, dbg)
            tiled_dbg_saved += 1

    # --- Draw overlay for every frame (reuses overlay_state) ---
    vis = frame.copy()

    # Draw GT box (optional)
    gt_box = GT.get(frame_id, None)
    if SHOW_GT_RECT and gt_box is not None:
        draw_box(vis, gt_box, (0,255,0), 2, "GT drone")

    # Draw last detection box (red)
    if overlay_state["last_box"] is not None and overlay_state["last_has_drone"]:
        draw_box(vis, overlay_state["last_box"], (0,0,255), 2, f"Pred conf={overlay_state['last_conf']:.2f}")

    # Draw guided ROI (cyan) if used on last inference
    if overlay_state["guided_rect_used"] is not None:
        draw_box(vis, overlay_state["guided_rect_used"], (255,255,0), 2, "Guided ROI (next inference)")

    # Draw motion ROI crops used (yellow)
    for r in overlay_state["motion_rois_rects"]:
        draw_box(vis, r, (0,255,255), 2, "Motion ROI crop")

    # Draw confirmation-guided ROI region (magenta)
    if overlay_state["confirm_guided_rect"] is not None:
        draw_box(vis, overlay_state["confirm_guided_rect"], (255,0,255), 2, "Confirmation guided ROI")

    # Continuity debug (optional)
    if SHOW_CONTINUITY_DEBUG and overlay_state["continuity_debug"] is not None and overlay_state["last_box"] is not None:
        info = overlay_state["continuity_debug"]
        if "prev_exp" in info:
            draw_box(vis, info["prev_exp"], (255,0,0), 2, "Expanded gate")
        if "pc" in info and "thr" in info:
            pc = info["pc"]; thr = info["thr"]
            cv2.circle(vis, (int(pc[0]),int(pc[1])), int(thr), (255,0,0), 2)
        if "pred" in info and "gate" in info:
            pred = info["pred"]; gatepx = info["gate"]
            cv2.circle(vis, (int(pred[0]),int(pred[1])), int(gatepx), (255,0,0), 2)

    # Text overlay
    y=22
    for line in overlay_lines(frame_id, t_s):
        cv2.putText(vis, line, (10,y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2, cv2.LINE_AA)
        y += 22

    if writer is not None:
        writer.write(vis)

    if SHOW_PREVIEW:
        cv2.imshow("benchmark", vis)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # per-frame csv row (use overlay_state for stable values)
    csv_rows.append([
        frame_id, t_s, is_infer,
        overlay_state["last_mode"], overlay_state["last_source"],
        overlay_state["last_has_drone"], overlay_state["last_conf"], overlay_state["last_box"],
        overlay_state["last_verified"], overlay_state["last_verify_mode"], overlay_state["last_yolo_calls"],
        gt_box,
        sum(1 for e in confirm_hits if e[0]),
        sum(1 for e in warning_hits if e[0]),
        confirmed_events, warning_events,
        no_drone_streak,
        counts["full_hit"], counts["guided_hit"], counts["motion_hit"], counts["verified_hit"]
    ])

    frame_id += 1

cap.release()
if writer is not None:
    writer.release()
if SHOW_PREVIEW:
    cv2.destroyAllWindows()

# Save per-frame csv
df = pd.DataFrame(csv_rows, columns=csv_cols)
df.to_csv(PER_FRAME_CSV, index=False)
print("Saved per-frame CSV:", PER_FRAME_CSV)
print("Saved video:", OUT_VIDEO_PATH)

# Summary metrics (inference frames only)
# Precision/Recall at IoU>=0.5
tp=fp=fn=0
for r in infer_eval:
    gb = r["gt_box"]
    pb = r["pred_box"]
    if gb is None and pb is None:
        continue
    if gb is None and pb is not None:
        fp += 1
    elif gb is not None and pb is None:
        fn += 1
    else:
        if iou_xyxy(pb, gb) >= 0.5:
            tp += 1
        else:
            fp += 1
precision = tp / max(1, tp+fp)
recall = tp / max(1, tp+fn)
ap50 = compute_ap50(infer_eval)

summary = {
    "run_id": RUN_ID,
    "video": os.path.basename(VIDEO_PATH),
    "infer_fps": INFER_FPS,
    "pre_motion_method": PRE_MOTION_METHOD,
    "use_motion": USE_MOTION_MINING,
    "use_guided_next": USE_GUIDED_ROI_NEXT_INFER,
    "verify_mode": VERIFY_MODE,
    "confirm_window": CONFIRM_WINDOW,
    "confirm_require": CONFIRM_REQUIRE,
    "warning_window": WARNING_WINDOW,
    "warning_require": WARNING_REQUIRE,
    "base_conf_full": BASE_CONF_FULL,
    "guided_roi_conf": GUIDED_ROI_CONF,
    "low_conf_detection": LOW_CONF_DETECTION,
    "precision_infer": precision,
    "recall_infer": recall,
    "ap50_infer": ap50,
    "avg_conf_infer": safe_mean(avg_conf_list),
    "avg_pre_ms": safe_mean(pre_ms_list),
    "avg_infer_ms": safe_mean(infer_ms_list),
    "avg_post_ms": safe_mean(post_ms_list),
    "confirmed_events": confirmed_events,
    "warning_events": warning_events,
}

print("\n=== SUMMARY (inference frames only) ===")
for k,v in summary.items():
    print(f"{k}: {v}")

# Append to master summary CSV
with open(MASTER_SUMMARY_CSV, "a", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow([
        summary["run_id"], summary["video"], summary["infer_fps"], summary["pre_motion_method"],
        summary["use_motion"], summary["use_guided_next"], summary["verify_mode"],
        summary["confirm_window"], summary["confirm_require"], summary["warning_window"], summary["warning_require"],
        summary["base_conf_full"], summary["guided_roi_conf"], summary["low_conf_detection"],
        summary["precision_infer"], summary["recall_infer"], summary["ap50_infer"],
        summary["avg_conf_infer"],
        summary["avg_pre_ms"], summary["avg_infer_ms"], summary["avg_post_ms"],
        summary["confirmed_events"], summary["warning_events"],
        ""
    ])
print("\nAppended run summary to:", MASTER_SUMMARY_CSV)

Video FPS: 29.970933447224024 Frames: 1351 Infer stride: 6
Saved per-frame CSV: video_benchmark_outputs\gopro_006__infer5fps__motion-None__verify-None__per_frame_log.csv
Saved video: video_benchmark_outputs\gopro_006__infer5fps__motion-None__verify-None.mp4

=== SUMMARY (inference frames only) ===
run_id: gopro_006__infer5fps__motion-None__verify-None
video: gopro_006.mp4
infer_fps: 5
pre_motion_method: None
use_motion: True
use_guided_next: True
verify_mode: None
confirm_window: 10
confirm_require: 9
warning_window: 10
warning_require: 9
base_conf_full: 0.25
guided_roi_conf: 0.4
low_conf_detection: 0.4
precision_infer: 0.3404255319148936
recall_infer: 0.2909090909090909
ap50_infer: 0.1824841824841825
avg_conf_infer: 0.6580467341428108
avg_pre_ms: 2.349524335977951
avg_infer_ms: 74.47989778714233
avg_post_ms: 28.52686017762511
confirmed_events: 1
warning_events: 0

Appended run summary to: video_benchmark_outputs\benchmark_runs_summary.csv


## 10) How many frames should motion ROI mining buffer?
In this notebook, motion mining buffers **inference frames** (`MOTION_BUFFER_STEPS`), not all video frames.

Practical guidance:
- **Static camera**: 3–7 inference frames is typically enough.
- **Trees/leaves**: increasing buffer helps only if you also use **persistence** (`MOTION_PERSIST_MIN_HITS`) so that random flicker is rejected.
- **Too many frames** can dilute fast targets (the drone moves, masks stop overlapping), so the persistence threshold must be tuned together with buffer length.

If `INFER_FPS=5`, buffering 5 inference frames covers ~1 second of time. For fast drones, you may prefer `MOTION_BUFFER_STEPS=3` with `MOTION_PERSIST_MIN_HITS=2`.
